In [6]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# PromptTemplate == string from template
# ChatPromptTemplate == message from template

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # temperature == for creating and randomizing the texts
)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?",
                                        )

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

# a = llm.predict("How many plants are there?")

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [10]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
# HumanMessage == We send a message to the AI
# AIMessage == The AI sends a message to us
# SystemMessage == We send a settings to the LLM, but it's not a question

template = ChatPromptTemplate.from_messages([
    # SystemMessage(content="You are a geography expert. And you only reply in {language}."),
    # AIMessage(content="안녕하세요. 제 이름은 {name}입니다."),
    # HumanMessage(content="What is the distance between {country_a} and {country_b}? Also what is your name?")
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "안녕하세요. 제 이름은 {name}입니다."),
    ("human", "What is the distance between {country_a} and {country_b}? Also what is your name?")  
])
# prompt_template == The communication template for the user and the LLMS
# So prompt is very important for the LLMs answering quaility 
prompt = template.format_messages(
    language="Korean",
    name="Jin",
    country_a="Mexico",
    country_b="Thailand"
)
chat.predict_messages(prompt)


AIMessage(content='안녕하세요! 저의 이름은 Jin입니다. 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다.')

In [14]:
# OutputParser == The communication parser for the user and the LLMs, LLM의 응답을 변형할 때가 있기 떄문에 필요하다. 
# 텍스트를 다른 자료구조로 변형하기

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()

p.parse("Hello, how, are, your")

# LangChain expression language == 

['Hello', 'how', 'are', 'your']

In [20]:
template = ChatPromptTemplate.from_messages([
    (
        "system", 
        "You are a list generating machine. Everything you are asked will be answered with a comma sepaerated list of max {max_items} in lowercase. Do NOT reply with anything else.", 
    ),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items = 10,
    question = "What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [21]:
# chat model, outputParser, template 
# create chain 

chain = template | chat | CommaOutputParser()
# .format_messages + chat.predict + .parse

# all = chain_one | chain_two | outputParser == langChain expression language

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

In [3]:
# chain_one | chain_two | outputParser == langChain expression language
# component = prompt [Dictionary type] | Retrieve | LLM [Single string, list of chat messages or a PromptValue] | Tool | OutputParser [The output of an LLM or ChatModel]
# Output Type = LLM = String | ChatModel = ChatMessage | Prompt = PromptValue | Retriever = List of documents | Tool = Depends on the tool | OutputParser = Depends on the parser
# | 는 파이프 라인처럼 앞에서 나온 컴포넌트의 결과를 뒤에 있는 것에 넣는다. 

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food. "),
])

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

chef_chain = chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain
# Runnable map = output key : chain / placeholder value for template output key

final_chain.invoke({
    "cuisine": "Indian"
})
# cuisine = prompt input for chef_chain
# recipe = output of chef_chain

# Streaming response == 응답을 작성하는 과정을 보여준다.

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Here's a recipe for a classic Indian dish called Butter Chicken:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust according to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt, to taste
- Fresh cilantro, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it becomes translucent.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute, stirring continuously.
4. Add the chicken pieces to the pan and cook until they are browned on 

AIMessageChunk(content="To make a vegetarian version of Butter Chicken, you can replace the chicken with a plant-based protein such as tofu or paneer. Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter (or vegan butter for a vegan version)\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup heavy cream (or coconut cream for a vegan version)\n- Salt, to taste\n- Fresh cilantro, for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it becomes translucent.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice